In [55]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("MinIO Example") \
    .config("spark.hadoop.fs.s3a.access.key", "minio_access_key") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio_secret_key") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .getOrCreate()


df_yellow_2023 = spark.read.parquet("s3a://processed/2023/yellow_tripdata_2023-01.parquet")

df_green_2023 = spark.read.parquet("s3a://processed/2023/green_tripdata_2023-01.parquet")

df_taxi_lookup = spark.read.csv("s3a://processed/taxi_lookup.csv")

df_yellow_2023.printSchema()
df_green_2023.printSchema()

# df_taxi_lookup.printSchema()

root
 |-- congestion_surcharge: double (nullable = true)
 |-- dolocationid: long (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pulocationid: long (nullable = true)
 |-- ratecodeid: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- vendorid: long (nullable 

In [22]:
df_yellow_2023.show()

+--------------------+------------+-------------------+------------------+-------------------+-----+-----------+---------------------+-------+---------------+------------+-------------------+------------------+-------------------+------------+----------+------------------+----------+------------+------------+-------------+--------+
|congestion_surcharge|dolocationid|   dropoff_datetime|  dropoff_latitude|  dropoff_longitude|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|    pickup_datetime|   pickup_latitude|   pickup_longitude|pulocationid|ratecodeid|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|trip_distance|vendorid|
+--------------------+------------+-------------------+------------------+-------------------+-----+-----------+---------------------+-------+---------------+------------+-------------------+------------------+-------------------+------------+----------+------------------+----------+------------+------------+-------------+--------

In [58]:
from pyspark.sql import functions as F

# Lấy dòng đầu tiên làm tên cột
header = df_taxi_lookup.first()  # Lấy dòng đầu tiên
columns = list(header.asDict().values())  # Chuyển dict_values thành danh sách

# Cập nhật tên cột cho DataFrame
df_taxi_lookup = df_taxi_lookup.toDF(*columns)

# Loại bỏ dòng đầu tiên
df_taxi_lookup = df_taxi_lookup.filter(df_taxi_lookup[columns[0]] != columns[0])

# Chuyển LocationID thành kiểu int
df_taxi_lookup = df_taxi_lookup.withColumn("LocationID", df_taxi_lookup["LocationID"].cast("int"))

# Hiển thị DataFrame sau khi cập nhật
df_taxi_lookup.show()


+----------+-------------+-----------------+------------+-----------+------------------+
|LocationID|      Borough|             zone|service_zone|   latitude|         longitude|
+----------+-------------+-----------------+------------+-----------+------------------+
|         1|          EWR|   Newark Airport|         EWR|40.68906405|-74.17725485035348|
|         2|       Queens|      Jamaica Bay|   Boro Zone| 40.6039936|       -73.8354124|
|         3|        Bronx|         Allerton|   Boro Zone|  39.915319|-87.93321507559128|
|         4|    Manhattan|    Alphabet City| Yellow Zone| 40.7251022|       -73.9795833|
|         5|Staten Island|    Arden Heights|   Boro Zone| 53.2843196|         -7.492801|
|         6|Staten Island|         Arrochar|   Boro Zone| 56.1954653|        -4.7480746|
|         7|       Queens|          Astoria|   Boro Zone| 46.1882007|      -123.8319802|
|         8|       Queens|     Astoria Park|   Boro Zone| 33.8241622|       -78.8918751|
|         9|       Qu

In [23]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time

from pyspark.sql import SQLContext
from pyspark.sql import types
from pyspark.sql.functions import isnan, when, count, col


%matplotlib inline
plt.style.use('ggplot')

We will be exploring the public data set that Google BigQuery has provided us. In particular we will be looking at the 2018 Yellow Taxi trips and the weather data set together. The variables from the weather data set are self explanatory from if you look at the field field names. The taxi data set however have some fields that are not immediately obvious in terms of their meaning, so the following are the detailed description of the fields associated with the taxi data set:
* **vendor_id**:	A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc
* **pickup_datetime**: The date and time when the meter was engaged.
* **dropoff_datetime**: The date and time when the meter was disengaged
* **passenger_count**: The number of passengers in the vehicle. This is a driver-entered value
* **trip_distance**: The elapsed trip distance in miles reported by the taximeter.
* **racodeId**: The final rate code in effect at the end of the trip.
* **store_and_fwd_flag**: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip
* **payment_type**:	A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip
* **fare_amount**: The time-and-distance fare calculated by the meter
* extra	Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and 1 rush hour and overnight charges.
* **mta_tax**: \$0.50 MTA tax that is automatically triggered based on the metered rate in use
* **tip_amount**:	Tip amount – This field is automatically populated for credit card tips. Cash tips are not included
* **tolls_amount**:	Total amount of all tolls paid in trip.
* **improvement_surcharge**:	\$0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
* **total_amount**:	The total amount charged to passengers. Does not include cash tips
* **pickup_latitude**: TLC latitude of Taxi Zone in which the taximeter was engaged 
* **pickup_latitude**: TLC longtitude of Taxi Zone in which the taximeter was engaged 
* **dropoff_latitude**: TLC latitude of Taxi Zone in which the taximeter was disengaged 
* **dropoff_longtitude**: TLC longtitude of Taxi Zone in which the taximeter was disengaged 
* **congestion_surcharge**: Total amount collected in trip for NYS congestion surcharge.
* **pulocationid**: TLC Taxi Zone in which the taximeter was engaged
* **dolocationid**: TLC Taxi Zone in which the taximeter was disengaged



* doanh thu theo từng ngày, giờ, location, của từng loại xe tính theo fare amount 
* các loại tiền khác (tips, extra, congestion_surcharge, mta_tax, improvement_surcharge)
* tìm số xe theo location xem location nào nhiều location nào ít
* só km đi của từng xe theo ngày của từng hãng xe
* phân tích tiền tip theo ngày theo giờ, và khu vực(pickup location)
* Tính phần trăm từng loại thanh toán 



* riêng data xanh:

* Top 10 người dùng hay đi xe xanh nhất, những người này hay đi vào những buổi nào, những ngày nào, những giờ nào, địa điểm ?
* Tìm số chuyến xe đi, tổng số tiền (fare amount), địa điểm hay đi(locationID) thời gian đi(buổi sang, trưa, tối) của một CustomerID

# Doanh thu theo từng ngày của từng loại xe tính theo fare amount 
# các loại tiền khác (tips, extra, congestion_surcharge, mta_tax, improvement_surcharge)

In [9]:
df_yellow_2023= df_yellow_2023.dropna()
df_green_2023= df_green_2023.dropna()

In [24]:
from pyspark.sql import functions as F

# Chuyển đổi cột pickup_datetime sang định dạng ngày
df_yellow_2023 = df_yellow_2023.withColumn("pickup_date", F.to_date("pickup_datetime"))

df_green_2024 = df_green_2023.withColumn("pickup_date", F.to_date("pickup_datetime"))


In [62]:
# Tính tổng doanh thu cho dữ liệu Yellow Taxi


def calculate_total_revenue(df_data):
    result = df_yellow_2023.groupBy("pickup_date").agg(F.sum("total_amount").alias("daily_revenue"))
    return result

daily_revenue_yellow =calculate_total_revenue(df_yellow_2023)

In [63]:
daily_revenue_yellow.show()

+-----------+------------------+
|pickup_date|     daily_revenue|
+-----------+------------------+
| 2023-01-01| 2061990.200000016|
| 2023-01-02|1851042.6300000404|
| 2023-01-11| 2505840.970000031|
| 2023-01-12| 2749092.620000016|
| 2022-10-24|            337.27|
| 2008-12-31|             80.55|
| 2023-01-05|2453593.5900000036|
| 2023-01-03|2244801.3800000367|
| 2023-01-13| 2789748.050000003|
| 2023-01-06|2429011.5400000424|
| 2023-01-04|2380041.5200000247|
| 2023-01-07|2391038.7100000107|
| 2022-10-25|            423.22|
| 2023-01-09| 2149560.000000049|
| 2023-01-10|  2373200.45000005|
| 2023-01-08| 2178832.640000033|
| 2022-12-31|            557.03|
| 2023-01-25| 2507019.940000066|
| 2023-01-21| 2462022.040000012|
| 2023-01-14| 2625746.270000053|
+-----------+------------------+
only showing top 20 rows



# tìm số xe theo location xem location nào nhiều location nào ít

In [64]:
# Đếm số xe theo pickup location

def count_vehicles_by_pickup_location(df, df_taxi_lookup):
    num_trips = (
        df
        .groupBy("pulocationid")
        .agg(F.count("*").alias("pickup_count"))
    )

    df_location_counts = (
        num_trips
        .join(df_taxi_lookup, num_trips.pulocationid == df_taxi_lookup.LocationID, "inner")
        .select(
            num_trips.pulocationid.alias("LocationID"),
            "pickup_count",
            "Borough",
            "zone",
            "service_zone",
            "latitude",
            "longitude"
        )
    )

    # Hiển thị kết quả
    return df_location_counts

df_location_counts = count_vehicles_by_pickup_location(df_yellow_2023, df_taxi_lookup)
df_location_counts.show()

+----------+------------+---------+--------------------+------------+------------------+------------------+
|LocationID|pickup_count|  Borough|                zone|service_zone|          latitude|         longitude|
+----------+------------+---------+--------------------+------------+------------------+------------------+
|        26|          82| Brooklyn|        Borough Park|   Boro Zone|         40.633993|       -73.9968059|
|        29|          76| Brooklyn|      Brighton Beach|   Boro Zone|       40.57964365|-73.96111022551311|
|        65|        1346| Brooklyn|   Downtown Brooklyn|   Boro Zone|40.691572050000005|-73.98676437548126|
|       191|         212|   Queens|      Queens Village|   Boro Zone|        40.7267692|       -73.7415208|
|       222|         139| Brooklyn|       Starrett City|   Boro Zone|        40.6478738|-73.87951922227273|
|       243|         290|Manhattan|Washington Height...|   Boro Zone|        35.2557439|       -80.8602626|
|        54|          26| Br

# Đếm số km đi của các xe theo từng ngày trong tháng

In [65]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType

def total_distance_per_day(df_data):
    df_distance_per_day = df_data.groupBy("pickup_date").agg(
        F.sum("trip_distance").alias("total_trip_distance")
    )

    return df_distance_per_day

df_distance_per_day = total_distance_per_day(df_yellow_2023)

df_distance_per_day.show()

+-----------+-------------------+
|pickup_date|total_trip_distance|
+-----------+-------------------+
| 2023-01-01| 354182.46000000427|
| 2023-01-02|  272698.6299999988|
| 2023-01-11|  300337.7000000006|
| 2023-01-12| 361010.24000000244|
| 2022-10-24|                0.0|
| 2008-12-31|              17.76|
| 2023-01-05| 318424.54000000126|
| 2023-01-03|  312183.1600000023|
| 2023-01-13| 344836.63999999943|
| 2023-01-06|  298130.4099999963|
| 2023-01-04|  305608.4299999988|
| 2023-01-07| 307703.70000000397|
| 2022-10-25| 10.780000000000001|
| 2023-01-09|  288794.9700000012|
| 2023-01-10| 287737.17999999796|
| 2023-01-08|  306056.7700000007|
| 2022-12-31|              72.33|
| 2023-01-25| 284225.83000000106|
| 2023-01-21|  304257.0500000035|
| 2023-01-14| 327036.81000000046|
+-----------+-------------------+
only showing top 20 rows



# số tiền (các loại tiền tips, tolls, fare, mta tax, total, congestion, extra) theo thời gian và địa điểm

In [80]:
def group_by_time_location(df, df_taxi_lookup, columns):

    # Nhóm theo ngày và tính tổng tip_amount
    df_per_day = df.withColumn("pickup_date", F.to_date(df["pickup_datetime"]))\
        .groupBy("pickup_date").agg(
            F.sum(columns).alias(f"total_{columns}"),
            F.avg(columns).alias(f"avg_{columns}")
        )

    # Nhóm theo giờ và tính tổng tip_amount
    df_per_hour = df.withColumn("pickup_hour", F.hour(df["pickup_datetime"]))\
        .groupBy("pickup_hour").agg(
            F.sum(columns).alias(f"total_{columns}"),
            F.avg(columns).alias(f"avg_{columns}")
        )

    # Nhóm theo khu vực và tính trung bình tip_amount
    df_per_location = df.groupBy("pulocationid").agg(
        F.sum(columns).alias(f"sum_{columns}"),
        F.avg(columns).alias(f"avg_{columns}")

    )

    # Nối với bảng taxi_lookup để lấy thông tin khu vực
    df_location = (
        df_per_location
        .join(df_taxi_lookup, df_per_location.pulocationid == df_taxi_lookup.LocationID, "inner")
        .select(
            df_per_location.pulocationid.alias("LocationID"),
            f"sum_{columns}", 
            f"avg_{columns}",  # Đổi tên thành avg_tip_amount để rõ ràng hơn
            "Borough",
            "zone",
            "service_zone",
            "latitude",
            "longitude"
        )
    )
    
    return df_per_day, df_per_hour, df_location



df_tip_per_day, df_tip_per_hour, df_location_tip_amounts = group_by_time_location(df_yellow_2023, df_taxi_lookup, 'tip_amount')

df_tip_per_day.show(2)
df_tip_per_hour.show(2)
df_location_tip_amounts.show(2)


df_total_amounts_per_day, df_total_amounts_per_hour, df_location_total_amounts = group_by_time_location(df_yellow_2023, df_taxi_lookup, 'total_amount')

df_total_amounts_per_day.show(2)
df_total_amounts_per_hour.show(2)
df_location_total_amounts.show(2)





+-----------+------------------+------------------+
|pickup_date|  total_tip_amount|    avg_tip_amount|
+-----------+------------------+------------------+
| 2023-01-01|240872.39999999377|3.5930189889466395|
| 2023-01-02| 215531.5699999948| 3.649119091155269|
+-----------+------------------+------------------+
only showing top 2 rows

+-----------+------------------+------------------+
|pickup_hour|  total_tip_amount|    avg_tip_amount|
+-----------+------------------+------------------+
|         12|479370.02000000945|3.1553693342636975|
|         22|474923.22999999224| 3.645739782601961|
+-----------+------------------+------------------+
only showing top 2 rows

+----------+------------------+-----------------+--------+--------------+------------+-----------+------------------+
|LocationID|    sum_tip_amount|   avg_tip_amount| Borough|          zone|service_zone|   latitude|         longitude|
+----------+------------------+-----------------+--------+--------------+------------+----

# Tính phần trăm từng loại hình phương thức thanh toán

In [72]:
from pyspark.sql import functions as F

def calculate_payment_type_percentage(df):
    # Áp dụng tên cho từng loại thanh toán dựa trên mã số
    df_with_payment_name = df.withColumn(
        "payment_type_name",
        F.when(df["payment_type"] == 1, "Credit card")
         .when(df["payment_type"] == 2, "Cash")
         .when(df["payment_type"] == 3, "No charge")
         .when(df["payment_type"] == 4, "Dispute")
         .otherwise("Unknown")  # Phòng trường hợp có mã ngoài 1, 2, 3, 4
    )
    
    # Nhóm theo payment_type_name và tính tổng số lượng từng loại thanh toán
    df_payment_type_count = df_with_payment_name.groupBy("payment_type_name").agg(
        F.count("payment_type_name").alias("payment_count")
    )
    
    # Tính tổng số lượng của tất cả các loại thanh toán
    total_payment_count = df_payment_type_count.agg(
        F.sum("payment_count").alias("total_count")
    ).collect()[0]["total_count"]
    
    # Tính phần trăm cho từng loại thanh toán
    df_payment_type_percentage = df_payment_type_count.withColumn(
        "payment_percentage", 
        (df_payment_type_count["payment_count"] / total_payment_count) * 100
    )
    
    return df_payment_type_percentage


df_payment_type_percentage = calculate_payment_type_percentage(df_yellow_2023)

df_payment_type_percentage.show()

+------------+-------------+------------------+
|payment_type|payment_count|payment_percentage|
+------------+-------------+------------------+
|           1|      2201009| 80.51916172615402|
|           3|        16962|0.6205181447231813|
|           2|       484771|17.734300290979917|
|           4|        30780|1.1260198381428794|
+------------+-------------+------------------+

